In [ ]:
# standard imports
import numpy as np
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt

# load up the saved dataset
import pickle
with open('../data/2_img_test.pkl', 'rb') as f:
    ds = pickle.load(f)

# add module to path so python can import it. This is the directory with 
# the __init__.py file in it to let python know this is a module.
import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/spicy_snow')

# import the function to test
from processing.s1_preprocessing import s1_amp_to_dB

# plot first time slice and VV band to see initial
ds['s1'].isel(time = 0).sel(band = 'VV').plot()
plt.show()
# snag the original time slice to compare afterwards
amp = ds['s1'].isel(time = 0).sel(band = 'VV').values.ravel()
amp = amp[amp != 0]
# run your newly written function over the dataset and capture returning dataset
s1_amp_to_dB(ds)
# plot first time slice and VV band to see dB change
ds['s1'].isel(time = 0).sel(band = 'VV').plot()
plt.show()
# grab the first time slice VV
dB = ds['s1'].isel(time = 0).sel(band = 'VV').values.ravel()
dB = dB[np.isfinite(dB)]
# assert the conversion was succesful
assert np.allclose(dB, 10 * np.log10(amp))

In [ ]:
ds = ds.set_xindex('relative_orbit')

In [ ]:
ds.sel(relative_orbit = 20)

In [ ]:
ds.where(ds.flight_dir == 'ascending', drop = True)